# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [ ]:
pip install opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilites

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)                # Make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections    

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmakrs
        draw_landmarks(image, results)
        
        #Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# len(results.left_hand_landmarks.landmark)

In [ ]:
results

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [ ]:
len(results.pose_landmarks.landmark)

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
extract_keypoints(results).shape

In [ ]:
468*3+33*4+21*3+21*3

# 4. Setup Folders for Collection

In [4]:
# Path for exproted data, numpy arrays
DATA_PATH =os.path.join('MP_DATA')
# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveu'])
# Thirty videos worth of data
no_sequences = 10
# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
# hello
## 0
## 1
## 2
## ...
## 29
# thanks

# I love you

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try: os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video lenght aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                # Draw landmakrs
                draw_landmarks(image, results)

                # NEW Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collection frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                     cv2.putText(image, 'Collection frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                     # Show to screen
                     cv2.imshow('OpenCV Feed', image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
label_map = {label:num for num, label in enumerate(actions)}

In [6]:
label_map

{'hello': 0, 'thanks': 1, 'iloveu': 2}

In [7]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [8]:
np.array(sequences).shape

(30, 30, 1662)

In [9]:
np.array(labels).shape

(30,)

In [10]:
X=np.array(sequences)
y=to_categorical(labels).astype(int)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 7. Build and Train LSTM Neural Network

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [13]:
log_dir = os.path.join('Logs')
tb_callbacks = TensorBoard(log_dir=log_dir)

In [55]:
model= Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [44]:
X.shape

(30, 30, 1662)

In [45]:
res = [0.7, 0.2, 0.1]

In [46]:
actions[np.argmax(res)]

'hello'

In [56]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
# epochs edit avilable
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callbacks])

Epoch 1/1000
1/1 [==============================] - 0s 117ms/step - loss: 0.0322 - categorical_accuracy: 1.0000
Epoch 2/1000
1/1 [==============================] - 0s 104ms/step - loss: 0.0317 - categorical_accuracy: 1.0000
Epoch 3/1000
1/1 [==============================] - 0s 58ms/step - loss: 0.0308 - categorical_accuracy: 1.0000
Epoch 4/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.0292 - categorical_accuracy: 1.0000
Epoch 5/1000
1/1 [==============================] - 0s 59ms/step - loss: 0.0288 - categorical_accuracy: 1.0000
Epoch 6/1000
1/1 [==============================] - 0s 57ms/step - loss: 0.0279 - categorical_accuracy: 1.0000
Epoch 7/1000
1/1 [==============================] - 0s 57ms/step - loss: 0.0268 - categorical_accuracy: 1.0000
Epoch 8/1000
1/1 [==============================] - 0s 58ms/step - loss: 0.0264 - categorical_accuracy: 1.0000
Epoch 9/1000
1/1 [==============================] - 0s 58ms/step - loss: 0.0255 - categorical_accuracy: 1.0000

KeyboardInterrupt: 

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

# 8. MakePredictions

In [48]:
res=model.predict(X_test)

In [49]:
actions[np.argmax(res[0])]

'hello'

In [50]:
actions[np.argmax(y_test[4])]

IndexError: index 4 is out of bounds for axis 0 with size 2

# 9. Save Model Weights

In [51]:
model.save('action.h5')

In [52]:
# del model

In [57]:
model.load_weights('action.h5')

# 10. Evaluation using a Confusing Matrix

In [58]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [72]:
yhat = model.predict(X_train)

In [73]:
yhat

array([[0.3653061 , 0.32020035, 0.3144936 ],
       [0.3538125 , 0.3236942 , 0.32249334],
       [0.35174367, 0.33147484, 0.31678146],
       [0.35896692, 0.32231104, 0.31872207],
       [0.33873022, 0.32808334, 0.33318642],
       [0.35230416, 0.32639927, 0.32129657],
       [0.35413086, 0.3224709 , 0.32339823],
       [0.35323355, 0.3275358 , 0.31923062],
       [0.35779127, 0.3203908 , 0.32181793],
       [0.35053775, 0.33190247, 0.31755975],
       [0.35178643, 0.3307097 , 0.31750384],
       [0.35336354, 0.32948074, 0.31715575],
       [0.35974386, 0.32375425, 0.31650195],
       [0.3548634 , 0.31901368, 0.32612288],
       [0.34930125, 0.3255337 , 0.3251651 ],
       [0.35566944, 0.31752637, 0.32680425],
       [0.3589092 , 0.32206044, 0.31903034],
       [0.35275614, 0.325411  , 0.32183287],
       [0.35817292, 0.33089593, 0.3109312 ],
       [0.3521651 , 0.32969883, 0.31813604],
       [0.3540184 , 0.3267936 , 0.31918797],
       [0.362724  , 0.3199037 , 0.31737226],
       [0.

In [74]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [75]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[ 0, 18],
        [ 0, 10]],

       [[20,  0],
        [ 8,  0]],

       [[18,  0],
        [10,  0]]], dtype=int64)

In [76]:
accuracy_score(ytrue, yhat)

0.35714285714285715

# 11. Test in Real Time